# 좀 더 디테일하게 모델을 활용하는 방법

이전 챕터에서는 기본 파라미터만 가지고

추론작업을 수행했습니다.

YOLO 모델에서 기본 제공하는 다양한 파라미터와 리턴값을 통해

애플리케이션을 만드는 것도 가능하지만,

좀 더 구체적으로 옵션을 조정하거나 접근하고 싶은 경우에는

아래 방식으로 코드를 짜기도 합니다.

주석을 읽어보면서 아래 코드와 실행결과를 파악해 보시기 바랍니다.

In [ ]:
"""
보안 관련 간단한 예시코드
실시간 프레임별로 웹캠에 보이는 사람 수를 세서
"연-월-일 시:분:초:밀리초 : 사람수" 포맷으로 
count_log.txt 파일 남기기
"""

from ultralytics import YOLO  # pip install ultralytics
import cv2  # pip install opencv-python
import datetime as dt  # 로그파일 작성을 위해 datetime 임포트

log_name = "count_log.txt"

# 모델 정의
model = YOLO("yolov8n.pt")

# 카메라 선택 및 설정
camera = cv2.VideoCapture(0)
camera.set(3, 640)  # 출력해상도(3:Width, 4:Height)
camera.set(4, 480)

while True:
    success, img = camera.read()
    results = model(img, show=True, save=True, stream=True,
                    project="infer", name="predict")
    # ↑ stream=True로 설정하면 list 대신 <generator> 반환

    for result in results:
        total_person_no = 0
        boxes = result.boxes
        for box in boxes:  # 각각의 박스 중
            cls = int(box.cls[0])
            if cls == 0:  # 클래스가 0(person)이면
                total_person_no += 1
        with open(log_name, "a") as f:
            f.write(f"{dt.datetime.now()} : {total_person_no}\n")

    if cv2.waitKey(30) == ord('q'):  # q 누르면 종료
        break

camera.release()
cv2.destroyAllWindows()

결과는 아래와 같습니다.

![](https://i.ibb.co/DW3DT5d/210.png)

이번엔 조금 더 복잡한 예제입니다.

일명, 추론결과에서 프레임별로 사람 수 세기 심화버전인데요.

result 안의 데이터 실시간으로 다뤄보기,

카메라 안에 사람이 있을 때에만

엑셀파일 로그 및 해당이미지 저장하기 등의 기능을 추가해보았습니다.

당장 이해가 되지 않아도 괜찮습니다. 

pandas의 DataFrame과 연동하여 로그를 저장하다가

추론이 종료되면 엑셀파일로 저장하는 코드입니다.


In [ ]:
%pip install pandas openpyxl ultralytics opencv-python

In [ ]:
import os
from time import sleep

from ultralytics import YOLO  # pip install ultralytics
import cv2  # pip install opencv-python
import pandas as pd  # pip install pandas
import datetime as dt

try:
    os.mkdir("log")
except FileExistsError:
    pass

start_time = dt.datetime.now()
df = pd.DataFrame()

camera = cv2.VideoCapture(0)
camera.set(3, 640)  # Width
camera.set(4, 480)  # Height

model = YOLO("yolov8n.pt")
classNames = model.names
# 기본 제공 레이블
# {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train',
#  7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign',
#  12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep',
#  19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella',
#  26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard',
#  32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard',
#  37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork',
#  43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange',
#  50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair',
#  57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv',
#  63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave',
#  69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase',
#  76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

while True:
    sleep(0.5)  # 0.5초당 한 컷씩만 추론하기
    _, img = camera.read()
    results = model(img, stream=True)
    # ↑ stream=True로 설정하면 <generator> 반환하므로 for문을 돌려야 함

    #
    for r in results:
        boxes = r.boxes  # 박스정보 꺼내서(이게 우리가 분석할 대상)
        # 직접 opencv로 박스도 그리고 레이블도 그려볼 것.
        # 기본창도 YOLO에서 말고, 아래에서 opencv창으로 띄울 예정.

        total_person_no = 0
        for box in boxes:
            # 바운딩박스 위치(좌상, 우하점 좌표) 추출
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # 소숫점 버림

            # 직접 웹캠이미지에 네모 그리기
            cv2.rectangle(img=img,
                          pt1=(x1, y1),
                          pt2=(x2, y2),
                          color=(255, 0, 255),
                          thickness=3)

            # 박스에서 신뢰도 추출
            confidence = box.conf[0]

            # 박스에서 클래스 추출
            cls = int(box.cls[0])

            # 주석 작성
            org = [x1, y1]  # 주석 작성할 좌표
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2
            cv2.putText(img, classNames[cls] + f" {confidence:.02f}", org, font, fontScale, color, thickness)

            if cls == 0:  # 박스의 클래스가 "person"일 때만 아래 작업 수행
                total_person_no += 1
                # 로그용 데이터 취합
                agg_data = [{"time": dt.datetime.now(),
                             "count": total_person_no,
                             "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                             "conf": f"{confidence:.02f}"}]
                df = pd.concat([df, pd.DataFrame(agg_data)], ignore_index=True)

        if total_person_no > 0:  # 화면에 사람이 있을 때만 이미지 저장
            cv2.imwrite(filename=f".q/{dt.datetime.now():%Y-%m-%d-%H%M%S}_{total_person_no}.jpg",
                        img=img)

    cv2.imshow(winname='Webcam', mat=img)  # opencv 창에 이미지 갱신
    if cv2.waitKey(1) == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()
df.to_excel(f"log_{start_time:%Y-%m-%d-%H-%M-%S}.xlsx", index=False)

위 코드를 실행하고 나면 아래와 같은 엑셀파일이 만들어져 있습니다.

![](https://i.ibb.co/jMqLLTj/211.png)

엑셀파일이나 데이터프레임을 통해서

보다 심도있는 데이터분석 작업을 수행할 수도 있겠습니다.

끝.

# 마치며

짧은 챕터였지만,

object detection 과 관련해서는 이 정도면 충분한 것 같습니다.

다음 챕터는 내 모니터화면을 소스삼아 추론하는 간단한 예제를 진행해보고

마치겠습니다.

수고하셨습니다!
